In [ ]:
import os 
import pandas as pd
import geopandas as gpd
from energyemissionsregio.config import SHP_PATH, DATA_PATH
from energyemissionsregio.disaggregation import perform_proxy_based_disaggregation
from energyemissionsregio.utils import solve_proxy_equation

In [ ]:
# Minimum capacities (MW), cumulative increase since 2020
planned_capacity_wind = 10000
planned_capacity_photovoltaic = 11800
planned_capacity_solar_thermoelectric = 600
planned_capacity_biomass = 380
planned_capacity_other = 60 # includes biogas, hydraulic, tidal, etc. 

### Onshore wind

In [ ]:
wind_potential = pd.read_csv(os.path.join("..", "data", "NECP_disaggregation", "proxies", "onshore_wind_potential.csv"))


In [ ]:

wind_potential["value"] = (wind_potential["_sum"] / wind_potential["_sum"].sum())
wind_potential["value"]  = wind_potential["value"] * planned_capacity_wind

final_df = wind_potential[["code", "value"]].copy()
final_df.rename(columns={"code": "region_code"}, inplace=True)

final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  f"planned_capacity_wind.csv"), index=False)

### Photovoltaic

In [ ]:
solar_potential = pd.read_csv(os.path.join("..", "data", "NECP_disaggregation", "proxies", "open_field_solar_thermal_potential.csv"))

In [ ]:
solar_potential["value"] = (solar_potential["_sum"] / solar_potential["_sum"].sum())
solar_potential["value"]  = solar_potential["value"] * planned_capacity_photovoltaic

final_df = solar_potential[["code", "value"]].copy()
final_df.rename(columns={"code": "region_code"}, inplace=True)

final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  f"planned_capacity_photovoltaic.csv"), index=False)

### Solar thermoelectric

In [ ]:
solar_potential["value"] = (solar_potential["_sum"] / solar_potential["_sum"].sum())
solar_potential["value"]  = solar_potential["value"] * planned_capacity_solar_thermoelectric

final_df = solar_potential[["code", "value"]].copy()
final_df.rename(columns={"code": "region_code"}, inplace=True)

final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  f"planned_capacity_solar_thermoelectric.csv"), index=False)

### Biomass

In [ ]:
nuts3_shp = gpd.read_file(os.path.join(SHP_PATH, "NUTS3.shp"))
nuts3_shp = nuts3_shp[nuts3_shp["code"].str.startswith("ES")].copy()

Agricultural residues

In [ ]:
# get agricultural biomass potential, which is at NUTS3 level 
agr_biomass_potential = pd.read_csv(os.path.join("..", "data", "NECP_disaggregation", "proxies", "agricultural_residues.csv"))

In [ ]:
agr_biomass_potential[agr_biomass_potential["code"].str.startswith("ES")]["value"].isna().any()

In [ ]:
agr_biomass_potential = agr_biomass_potential[agr_biomass_potential["code"].str.startswith("ES")][["code", "value"]].copy()

In [ ]:
agr_biomass_potential = agr_biomass_potential.groupby("code").sum().reset_index()

In [ ]:
# ensuring no NAs in the data
final_df_agr = pd.merge(nuts3_shp, agr_biomass_potential, on="code", how="left")
final_df_agr.isna().any()

In [ ]:
final_df_agr = final_df_agr[["code", "value"]].copy()
final_df_agr.rename(columns={"code": "region_code"}, inplace=True)
final_df_agr["value_confidence_level"] = 5 # VERY HIGH

In [ ]:
# disaggregate to LAU 
proxy_data = pd.read_csv(os.path.join("..", "data", "imputed_data", "es_utilized_agricultural_area.csv"))

In [ ]:
proxy_data = proxy_data[proxy_data["region_code"].str.startswith("ES")].copy()

In [ ]:
final_df_agr_lau = perform_proxy_based_disaggregation(final_df_agr, proxy_data, source_resolution="NUTS3", proxy_confidence_level=4)


In [ ]:
final_df_agr_lau

Forest residues

In [ ]:
# get forest biomass potential, which is at NUTS3 level 
fst_biomass_potential = pd.read_csv(os.path.join("..", "data", "NECP_disaggregation", "proxies", "forest_residues.csv"))

In [ ]:
fst_biomass_potential["source"].unique()

In [ ]:
fst_biomass_potential[fst_biomass_potential["code"].str.startswith("ES")]["value"].isna().any()

In [ ]:
fst_biomass_potential = fst_biomass_potential[fst_biomass_potential["code"].str.startswith("ES")][["code", "value"]].copy()

In [ ]:
# ensuring no NAs in the data
final_df_fst = pd.merge(nuts3_shp, fst_biomass_potential, on="code", how="left")
final_df_fst.isna().any()

In [ ]:
final_df_fst = final_df_fst[["code", "value"]].copy()
final_df_fst.rename(columns={"code": "region_code"}, inplace=True)

final_df_fst["value_confidence_level"] = 5 # VERY HIGH

In [ ]:
# disaggregate to LAU 
proxies = ["broad_leaved_forest_cover", "coniferous_forest_cover", "mixed_forest_cover", "natural_grasslands_cover", "moors_and_heathland_cover", "sclerophyllous_vegetation_cover", "transitional_woodland_shrub_cover"]
proxy_data_dict = {}
for proxy in proxies:
    proxy_data = pd.read_csv(os.path.join(DATA_PATH, f"{proxy}.csv"))
    proxy_data = proxy_data[proxy_data["region_code"].str.startswith("ES")][["region_code", "value"]].copy()
    proxy_data["value_confidence_level"] = 5 # VERY HIGH
    proxy_data_dict[proxy] = proxy_data

In [ ]:
proxy_equation = "broad_leaved_forest_cover + coniferous_forest_cover + mixed_forest_cover + natural_grasslands_cover + moors_and_heathland_cover + sclerophyllous_vegetation_cover + transitional_woodland_shrub_cover"
solved_proxy_df = solve_proxy_equation(proxy_equation, proxy_data_dict)

In [ ]:
solved_proxy_df

In [ ]:
final_df_fst_lau = perform_proxy_based_disaggregation(final_df_fst, solved_proxy_df, source_resolution="NUTS3", proxy_confidence_level=4)

In [ ]:
final_df_fst_lau["value"].sum()

disaggregated NECP biomass potential

In [ ]:
biomass_potential = pd.merge(final_df_agr_lau, final_df_fst_lau, on="region_code")

In [ ]:
biomass_potential["value"] = biomass_potential["value_x"] + biomass_potential["value_y"] 

In [ ]:
biomass_potential["share"] = (biomass_potential["value"] / biomass_potential["value"].sum())
biomass_potential["value"]  = biomass_potential["share"] * planned_capacity_biomass

final_df = biomass_potential[["region_code", "value"]].copy()

final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  f"planned_capacity_biomass.csv"), index=False)

### Other

In [ ]:
population = pd.read_csv(os.path.join(DATA_PATH, "population.csv"))
population = population[population["region_code"].str.startswith("ES")]

In [ ]:
population

In [ ]:
population["share"] = (population["value"] / population["value"].sum())
population["value"]  = population["share"] * planned_capacity_other

final_df = population[["region_code", "value"]].copy()

final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  f"planned_capacity_other_renewables.csv"), index=False)